In [ ]:
%%capture
!pip install transformers datasets sentencepiece sacrebleu

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Wasserstoff-AI/legalTransEn_Indic")


README.md:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

data.jsonl:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/89951 [00:00<?, ? examples/s]

In [ ]:
model_name = "Helsinki-NLP/opus-mt-hi-en"

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/304M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def preprocess(example):
    input_text = example["hindi"]
    target_text = example["english"]
    model_inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_text, truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset["train"].map(preprocess, batched=True)


Map:   0%|          | 0/89951 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
import torch

def get_token_bias_mask(input_ids, tokenizer, boost_tokens, boost_value=2.0):
    mask = torch.zeros_like(torch.tensor(input_ids))
    for token in boost_tokens:
        token_id = tokenizer.convert_tokens_to_ids(token)
        mask[input_ids == token_id] = boost_value
    return mask


In [ ]:
from transformers.models.marian.modeling_marian import MarianAttention

from transformers.models.marian.modeling_marian import MarianAttention
import torch.nn.functional as F

class LegalBiasMarianAttention(MarianAttention):
    def forward(
        self,
        hidden_states,
        key_value_states=None,
        past_key_value=None,
        attention_mask=None,
        layer_head_mask=None,
        output_attentions=False,
    ):

        outputs = super().forward(
            hidden_states,
            key_value_states=key_value_states,
            past_key_value=past_key_value,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            output_attentions=output_attentions,
        )


        if hasattr(self, "token_bias_mask") and self.token_bias_mask is not None:

            attn_weights = outputs[1] if output_attentions else None
            if attn_weights is not None:
                attn_weights += self.token_bias_mask.unsqueeze(1).unsqueeze(2).to(attn_weights.device)

        return outputs



In [ ]:
from transformers.models.marian.modeling_marian import MarianEncoder

boost_tokens = [
    "▁समझौता", "▁पक्ष", "▁होगा", "▁प्रस्ताव", "▁स्वीकृति", "▁दावा", "▁उत्तरदायित्व", "▁शर्त",
    "▁नुकसान", "▁उल्लंघन", "▁विवाद", "▁समझौते", "▁न्यायालय", "▁अधिकार", "▁उत्तराधिकारी",
    "▁हित", "▁अनुमति", "▁त्याग", "▁उपाय", "▁क्षेत्राधिकार", "▁स्थल", "▁सुनवाई", "▁याचिका",
    "▁आवेदन", "▁प्रतिज्ञापत्र", "▁प्रार्थनापत्र", "▁अस्थायी", "▁स्थायी", "▁निषेधाज्ञा",
    "▁न्याय", "▁कानून", "▁दायित्व", "▁धारा", "▁उपबंध", "▁संशोधन", "▁समाप्ति", "▁हस्तांतरण",
    "▁प्रकटीकरण", "▁पालन", "▁गोपनीयता", "▁विश्वास", "▁बांड", "▁प्रतिभूति", "▁ऋणदाता",
    "▁देयता", "▁विश्वासी", "▁लाभार्थी", "▁मध्यस्थता", "▁सुलह", "▁कार्यान्वयन", "▁बल",
    "▁प्राकृतिक", "▁पूर्वाग्रह", "▁रिहाई", "▁जप्ती", "▁जुर्माना", "▁दंड", "▁अपील",
    "▁प्रतिदावा", "▁साक्ष्य", "▁गवाही", "▁खोज", "▁लापरवाही", "▁प्रतिनिधित्व", "▁वारंटी",
    "▁विशेष", "▁प्रदर्शन", "▁भौतिक", "▁समाप्ति", "▁नवीनीकरण", "▁धोखाधड़ी", "▁दबाव",
    "▁प्रतिषेध", "▁बंधक", "▁पट्टा", "▁किरायेदार", "▁मालिक", "▁कब्जा", "▁स्वामित्व",
    "▁विभाज्यता", "▁विवेकाधिकार", "▁चोट", "▁लेखांकन", "▁अधिकारिता", "▁शिकायत", "▁खरीद",
    "▁निविदा", "▁वारंट", "▁समन", "▁गिरफ़्तारी", "▁धरोहर", "▁जमानत", "▁संपत्ति", "▁उत्तरदायित्व",
    "▁दायित्व", "▁अधिप्राप्ति", "▁भुगतान", "▁वचन", "▁प्रतिज्ञा", "▁प्रतिपूर्ति"
]


for layer in model.model.encoder.layers:
    layer.self_attn = LegalBiasMarianAttention(
        embed_dim=layer.self_attn.embed_dim,
        num_heads=layer.self_attn.num_heads,
        dropout=layer.self_attn.dropout,
        is_decoder=False,
    )


In [ ]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader

class LegalBiasTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

        input_ids = inputs["input_ids"]
        attention_bias = get_token_bias_mask(
            input_ids=input_ids,
            tokenizer=tokenizer,
            boost_tokens=boost_tokens,
            boost_value=2.0
        )
        for layer in model.model.encoder.layers:
            layer.self_attn.token_bias_mask = attention_bias

        return super().compute_loss(model, inputs, return_outputs)



In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./marian-legal-hi",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=3,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
)



In [ ]:
trainer = LegalBiasTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


<ipython-input-10-afad97fce4f4>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `LegalBiasTrainer.__init__`. Use `processing_class` instead.
  trainer = LegalBiasTrainer(
<ipython-input-5-e3bb3416b2bf>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.zeros_like(torch.tensor(input_ids))


Step,Training Loss


In [ ]:
model.save_pretrained("./marian-legal-hi-final")
tokenizer.save_pretrained("./marian-legal-hi-final")


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IndicTransToolkit import IndicProcessor
# recommended to run this on a gpu with flash_attn installed
# don't set attn_implemetation if you don't have flash_attn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
model_name = "ai4bharat/indictrans2-en-indic-dist-200M"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
).to(DEVICE)

ip = IndicProcessor(inference=True)

input_sentences = [
    "When I was young, I used to go to the park every day.",
    "We watched a new movie last week, which was very inspiring.",
    "If you had met me at that time, we would have gone out to eat.",
    "My friend has invited me to his birthday party, and I will give him a gift.",
]

batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)


inputs = tokenizer(
    batch,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)


with torch.no_grad():
    generated_tokens = model.generate(
        **inputs,
        use_cache=True,
        min_length=0,
        max_length=256,
        num_beams=5,
        num_return_sequences=1,
    )


with tokenizer.as_target_tokenizer():
    generated_tokens = tokenizer.batch_decode(
        generated_tokens.detach().cpu().tolist(),
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True,
    )


translations = ip.postprocess_batch(generated_tokens, lang=tgt_lang)

for input_sentence, translation in zip(input_sentences, translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")
